In [12]:
using GR
using Random
import Dates

In [13]:
#### specify path of Julia Code for 2D Models ####
path = "/home/jakob/Dokumente/Repositories/Modeling_Framework/2D_Models/"


##### include 2D Modelling Framework ####
include(string(path, "./2D_model_library.jl"));


########## OPERATIONAL ##########
##### how to chose the actual velocity? #####

#op_model = "Constant"
#op_model = "Collision_Free_Speed"
#op_model = "RVO"
#op_model = "TTC_CostFunction"
#op_model = "SocialForce"
op_model = "AV_Model"
#op_model = "Intrusion_Force"

include_op_model(op_model, path)

########## TACTICAL ##########
#### how to chose the preferred velocity? ###

#tact_model = "Constant"
tact_model = "IN_Model"

include_tact_model(tact_model, path)


########## STRATEGIC ##########
### how to chose the desired velocity ###

strat_model = "Constant"
#strat_model = "Towards_Goal"

include_strat_model(strat_model, path)


##### which update scheme? #####

update = "Parallel_Update"
#update = "Step_Update"

include_update(update, path)


##### which order? i.e. Acceleration or Velocity based? ####
#order = "first"
order = "second"

include_order(order, path);

In [14]:
#### parameters of the agents ####
p = [1.6, 1.2, 3.0, 1.0, 0.3, 0.0, 0.0, 0.1, 0.05, 0.0, 0.0, 0.3, 0.6283185307179586]
p_desc = "v_max, v_des, T, T2, l, step_time, τ_A, τ_R, α, ζ_h, ζ_v, r_priv, ϕ"

### of the system ###
N = 8

system_size = (20.0, 20.0) #important for periodic boundaries

### of the simulation ###

# simulation time-step #
δt = 0.01
sim_p = [0.0, 20.0, 0.05, δt, p[1]*p[2]] 
sim_p_desc = "t_relax, t_max, dt_save, dt, r"


### initialize geometry ###
geometrie = create_geometry_single_obstacle((37.5,31.5), 0.3)
#geometrie = create_geometry_closed_room(10.0, 10.0, 0.3)


### initialize crowd ###
menge = create_crowd(N, geometrie)

Init_Hom_Parameters!(p, menge)
Init_Hom_Vels!(menge, 1.0)

R_ = 5.0
p0 = (10.0, 10.0)
Init_Antipodal!(menge, R_, p0)

In [15]:
positions, headings, ttcs = Simulate!(menge, geometrie, sim_p[1], sim_p[2], sim_p[3], sim_p[4], sim_p[5], system_size);

In [16]:
GR.inline("mov")

j = 1

dt_save = sim_p[3]
geometry_x, geometry_y = rewrite_geometry(geometrie)
boundaries_x, boundaries_y = boundaries_rectangle_positions(system_size)

while j*sim_p[3] < sim_p[2]-sim_p[1]
    
        GR.clearws()

        GR.setwindow(4,16, 4, 16)
        GR.setviewport(0.0, 1, 0.0, 1)       

        plot_agents(positions[j, :], headings[j, :], 2.1, 100, 0.7, 999, 0.1)


        #model_description_1 = string("Direction Model = ", direction_model) 
        #GR.text(0.1, 0.95, model_description_1)
    
        #model_description_2 = string("Velocity Model = ", velocity_model) 
        #GR.text(0.1, 0.9, model_description_2)
    
        timer = string("time ", round(j*dt_save, digits = 1), "s")
        GR.text(0.8, 0.95, timer)
        
        GR.updatews()
    
        j = j+1
end

In [17]:
GR.show()

GR.HTML("<video autoplay controls><source type=\"video/mp4\" src=\"data:video/mp4;base64,AAAAFGZ0eXBxdCAgAAACAHF0ICAAAAAId2lkZQAI4UFtZGF0AAACsQYF//+t3EXpvebZSLeWLNgg2SPu73gyNjQgLSBjb3JlIDE2MyAtIEguMjY0L01QRUctNCBBVkMgY29kZWMgLSBDb3B5bGVmdCAyMDAzLTIwMjEgLSBodHRwOi8vd3d3LnZpZGVvbGFuLm9yZy94MjY0Lmh0bWwgLSBvcHRpb25zOiBjYWJhYz0xIHJlZj0zIGRlYmxvY2s9MTowOjAgYW5hbHlzZT0weDM6MHgxMTMgbWU9aGV4IHN1Ym1lPTcgcHN5PTEgcHN5X3JkPTEuMDA6MC4wMCBtaXhlZF9yZWY9MSBtZV9yYW5nZT0xNiBjaHJvbWFfbWU9MSB0cmVsbGlzPTEgOHg4ZGN0PTEgY3FtPTAgZGVhZHpvbmU9MjEsMTEgZmFzdF9wc2tpcD0xIGNocm9tYV9xcF9vZmZzZXQ9LTIgdGhyZWFkcz0xMiBsb29rYWhlYWRfdGhyZWFkcz0yIHNsaWNlZF90aHJlYWRzPTAgbnI9MCBkZWNpbWF0ZT0xIGludGVybGFjZWQ9MCBibHVyYXlfY29tcGF0PTAgY29uc3RyYWluZWRfaW50cmE9MCBiZnJhbWVzPTMgYl9weXJhbWlkPTIgYl9hZGFwdD0xIGJfYmlhcz0wIGRpcmVjdD0xIHdlaWdodGI9MSBvcGVuX2dvcD0wIHdlaWdodHA9MiBrZXlpbnQ9MjUwIGtleWludF9taW49MjQgc2NlbmVjdXQ9NDAgaW50cmFfcmVmcmVzaD0wIHJjX2xvb2thaGVhZD00MCByYz1hYnIgbWJ0cmVlPTEgYml0cmF0ZT00MDAwIHJhdGV0b2w9MS4wIHFjb21wPTAuNjAgcXBtaW49MCBxcG1heD02OSBxcHN0ZXA9NCBpcF9yYXRpbz0xLjQwIGFxPTE6MS4wMACAAAALfGWIhAAY//73ye/AputbW7W/k89I/Cy3PsIqP25bE7TqAAADAAADAAAML8iPv5qL+4vFn/6YAAAT39i6vD8pyABy6980/j5NL8Dw/OuxCstfN9/wxeF/uRxQKUmIRkbXUVsKsp7ylnycdBU6JN1XJZWIwFrFbef+gshK/kPRmW8IpeumHbGdztz2fmGGnOejMUc+GJGcAXqxVlulIRSPcykGWOEL5qCd87rKkrDyLyh/+yFbuKr1zNtqLxyNuDFFqkWKBERCB2C8hv0q69aWnjEqd75gjwRrs+jdHkBbYCYvASUQdTp1R4ljDBSOIJ4pzUCSDkWh4jh1pjRQ/AWXmbU7vtGiwTv9T1SmYj/ArKkE34tRPuFRJzDMRwYnrKpmwh9BwG+gzDH20TTr1aZglwqestCsAvAsCu92xXA0NUrIbuhKrnJDeUr+i4vRXShXeKyiAnY0VBFWUqlA/t6Vho9FdjodvdkMX/8j26BetPAfnpsZM2i7sSQRk5t9Fur4GZ8z9wJnNYTU4HrzYFa2wq8fXSHQVcGfS9dhZTeZPV2XQK9BjJanwJHyW3X2pla/gvmYkjq380rWE55KNCCd5/pt3XeF6zGS6q79/ItdLua4NdZ8yJ6O9H3fTfRc6TBjyaDZ+lX8wE/xhxcOUc7HqZFp1u3dbBA9lJbIF9EHp7UBa6P3YyzlRyPVV8XJACmQqPIaaGzdmiRAe5aDFsSfCLBRj6bNLbUi5XTdUsml4cmK2ywNifKDj12dVwh+W9XIBxDObtX1PDyWdQ8Nvd6/Lx3W/mfEeFeRFToJf2rxjgaiRax2rBeqKvWyzzYJf+p+aLGR0LU3/jlBokI27yObrjwwezfszbKMqUt/iWeylhPZ7i5cb++tIX3aPRH/YD5kphSbUMpsl3hsYH3a5ek5K7yjKuRXzxajD9+SKdeq0psbfZ6E9fMA+IRPhjz7AtAZMIE5Hn8SebHOEdR3R992Bxy0SZy4PJHXWtlDHTtUT11EdgXYtxbppcWun3PHzDujxdEplQNHWpxNei83uuD06htYOnq6cUv+WfFBcWmDI46xi3y6cds7VkMAA4DKqqf+9QSTReG0Clv5s/gPWRL9JrHEY9PfeZleni8kyNZlSLJvz9vJf/UqEBcEo2/sjx8rXQEq1k7Zm8seYHno2vrude791A2wK4R96Vdo3s0uf8/7SextyDCQxAlUNrIkAS0kCOEvOpf5ImxPjOtrTOXAczCM6T0MdatglRdN8031zl+BUeSDSpd6mAZTF+NqbwnCebT4pz4s4rSPXkPwWxpG70yDKL6qcoeyqYnggskCVg/ciZ2prw5sHLAmkn12m+0DDyIoFhS+QrlPVUu6Zc8U2NOqq+FCdziQGo83t0TZsHcw5jgP89Xv/1LSCQOG7P+Yk4zIp9xCG+Z8wI7mrUaTQyZK3U/ZSn+fT/bXN6ZhCr+X5F/UDOmVJn6T20DZMd9dn9w7ednD9nh/uO7ZoAM7ttrselhFdapIUjNNc1PtYBPrBZyaHsuzNMN8YsVEmJa1q88HBUuCJLLg545DSwLqUTHwfJt3021ZSPkgw9tkRksyXdlrIpMqpWcgkYYu/GMnozeeiQ3j/ad6BmwaG/fpA0frrYFjqLSRFI+hN/PFuO24b/3L9yh128zFBNsporKy1+7xoLMGnRanp6ULDmKcVjjI02dpBGdUnF1sC9gKt8b0sPf7i6sIoQk1SuqFAUKu4TosUsRXsyLg1AIlL9h166bsBgOeIVYiyJotpLMq50Uo//WofqR76I7HK2SIn0fKranv4OituLGmbgu1Ey9YXzt4Xxj/+3sreOGsLU/KNU/WbQDufpogLhEDTmAiJY9bLwU+4pnAtZ42r45wG60861QQz0/WnJZRSIiyc1kaf/pgx7RPk5sVvDL2uScJ+u9MXP5JEd2jE3kcp547t0m8gi63LEvcxx+eZXM6h136FNjclilrG7bCFZguN0sE9lh9WAWFyEt2MtKq5Tk5XeEkdCYIUOy/n6SKQrVhupgp6n9MWNym0MyLZmnM3xkZKhm7Kb/2hw5l82jYPQ5PagstZJ8zt+XCPWg/x2xlHxBLk4UUx6OGflEUsJYWsAQKDwTgJglmQUe3TgX5l36CtPk23UKMpnHB8F0w+K8oQOhKJZZi4bQ20rDwng0mCyIpygerqDJDvxhxQ7odH5RPsRJEOgD39Q9f41nRVwojaJUkb2ovTZS3fAAfbRIlXxgidnNgg+IRe7cUT5/KdzL3PXguKty9CDRCIAAAAwAAAwAAAwAAAwAKCANwtV3HCmqc3aJphT5PU/bBGKs0/SDaDeSQzGFb6AK2VLBf1Q9UaT3dBJy8m1JBg29ZKOw2tdjlQwiUBxw7qkk1A+31qXCe9WcJv75HT9/Lm811SH1zExcmUA+fNsVjuuEaqfhOS5wO9EINzs35uERGVmHoPFbTCmWsE+2YHYdWuu1yRn+o3bmcDZbjZH6BDlQm8QxcbdgQk9vxWw5ma2Lg6nuNWEweDWQ2vQK6BlLedYR8unxyiL5Bm8T9l0NH9dZFkcjKXzQB8GjcgKi/qgAJdV9RuCz6C0djquHTgfCF7Ug74EFeXavyTeA+ee05UeGkQLvuPuc8AcPFM//I355C4+Uf7VNuc5+wIv2Wte18/tOlc+7GjlO0xUE0O+oDE4SmQoAD1NA6KIpnEB/sbLGddB4FFHM24u/vYBysZ1i5ffhQI2ppINBr2mTirYj741r+mL9DnM5R/BI/nlLoqb/gJEYrBUVydtWqM9HmSl56GjTWKECh69OrBgSKbJamQhMKeWB26XJEWm/ZlCyZ5c+7tdf9efhb3ZPi3L/wNRWo/EcwHtCDSnkJDSh3O+EY87VwAYf/eIMJvLnclyWV7lH/e5YBkVriGCo3U2g2+y1iBiKwAAADAAADAAADAAADAAADAAA

In [84]:
### To save the results in a .txt file in the format ID/Frame/X/Y ###

Path = "/home/jakob/Dokumente/Sim_Data/"
dir = "Circle_8_AV_IN"
header = Header(p, p_desc, sim_p, sim_p_desc, op_model, tact_model, order, update, N, system_size);

Save_Data!(Path, dir, header, positions)

In [ ]:
### Perform Multiple Runs and Save the Output ###

samples = 15

for i in 1:samples
    
    Random.seed!()
    Init_Hom_Vels!(menge, 0.0)
    
    β, D, x_min, y_min, ϵ = pi, i/3, 2.5, 5.0, 0.00
    Init_Two_Agents_α!(menge, β, D, x_min, y_min, ϵ)

    
    positions, headings, ttcs = Simulate!(menge, geometrie, sim_p[1], sim_p[2], sim_p[3], sim_p[4], sim_p[5], system_size);
    Save_Data!(Path, dir, header, positions)
    
end

In [ ]:
### Save Initial Conditions for UMANS Software ###

name = "Circle_8"
path = "/home/jakob/Dokumente/Renne_Software/UMANS_3/examples/agents/"


#Set_Goal_from_E_des!(menge, (99.0, 0.0))
#Change_Coordinate_System_Agents!(menge, -1 .*(system_size./2))
Write_Agents_XML(menge, name, path)

In [ ]:
### Save Scenario for UMANS Software ###

path = "/home/jakob/Dokumente/Renne_Software/UMANS_3/examples/"
policy = "SocialForces"
agents = "Circle_8"

name = string(agents, policy)

Write_Scenario_XML(path, name, sim_p, system_size, policy, agents)